# AHAB app Functions

In this notebook I will be testing the different functions needed to build up the app

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import json

In [16]:
with open('../data/CLEAN/audio_df.json', 'r') as audio:
    audio_products = json.load(audio)

with open('../data/CLEAN/camera_df.json', 'r') as cameras:
    camera_products = json.load(cameras)

with open('../data/CLEAN/lens_df.json', 'r') as lenses:
    lens_products = json.load(lenses)

with open('../data/CLEAN/lights_df.json', 'r') as lights:
    light_products = json.load(lights)

with open('../data/CLEAN/rental_places.json', 'r') as places:
    rental_places = json.load(places)

In [28]:
for i in range(len(rental_places)):
    print(rental_places[i]['rental_place_name'])

WELAB
RC SERVICE
OVIDE


In [17]:
audiodf = pd.DataFrame(audio_products)
camerasdf = pd.DataFrame(camera_products)
lensesdf = pd.DataFrame(lens_products)
lightsdf = pd.DataFrame(light_products)
rentaldf = pd.DataFrame(rental_places)

In [55]:
lightsdf

,category,subcatgory,type,brand,name,price_a_day,link,rental_place_id,product_id,tokens,matched_id
0,lighting,bastidores,telas-y-bastidores,avenger,kit x3 banderas hollywood avenger,12,https://welabplus.com/shop8/iluminacion/telas-...,1,1,"[kit, x3, banderas, hollywood, avenger]",1
1,lighting,bastidores,telas-y-bastidores,avenger,bandera hollywood avenger,6,https://welabplus.com/shop8/iluminacion/telas-...,1,2,"[bandera, hollywood, avenger]",2
2,lighting,bastidores,telas-y-bastidores,avenger,kit x2 bastidores abatibles 24×36 cm,12,https://welabplus.com/shop8/iluminacion/telas-...,1,3,"[kit, x2, bastidores, abatibles, 24×36, cm]",3
3,lighting,sujecion,accesorios-iluminacion,lastolite,fondo croma-key plegable verde y azul,20,https://welabplus.com/shop8/iluminacion/acceso...,1,4,"[fondo, croma-key, plegable, verde, y, azul]",4
4,lighting,sujecion,accesorios-iluminacion,unknown,estico 1×1,2,https://welabplus.com/shop8/iluminacion/acceso...,1,5,"[estico, 1×1]",5
...,...,...,...,...,...,...,...,...,...,...,...
89,iluminacion,trípode,accesorios,avenger,trípode iluminación avenger century 2030d,pedir presupuesto,https://www.ovide.com/alquiler-es/tripode-foco...,3,37,"[trípode, iluminación, avenger, century, 2030d]",90
90,iluminacion,trípode,accesorios,avenger,trípode iluminación avenger combo 10 acero,pedir presupuesto,https://www.ovide.com/alquiler-es/tripode-foco...,3,38,"[trípode, iluminación, avenger, combo, 10, acero]",91
91,iluminacion,trípode,accesorios,avenger,trípode iluminación con ruedas avenger a5017,pedir presupuesto,https://www.ovide.com/alquiler-es/avenger-a5017/,3,39,"[trípode, iluminación, con, ruedas, avenger, a...",92
92,iluminacion,trípode,accesorios,avenger,trípode iluminación con ruedas avenger a5033,pedir presupuesto,https://www.ovide.com/alquiler-es/tripode-rued...,3,40,"[trípode, iluminación, con, ruedas, avenger, a...",93


In [8]:
products_dataframes = [audiodf, camerasdf, lensesdf, lightsdf, rentaldf]

In [51]:
user_input_list = input('Feed me your needs: ').split(',')

In [52]:
user_input_list

['df hsfve jsdb', ' jhdbhsjfbsd', ' njsdbjf. jdsbfhjbds']

In [54]:
for i in range(len(user_input_list)):

    print(user_input_list[i].split(' '))

['df', 'hsfve', 'jsdb']
['', 'jhdbhsjfbsd']
['', 'njsdbjf.', 'jdsbfhjbds']


In [ ]:
# Function for matching products by token

def find_matching_products(user_input_tokens, products_df):
    
    matching_products = []
    
    for idx, row in products_df.iterrows():
        product_tokens = row['tokens']  # Tokens from product DataFrame
        
        # Perform fuzzy matching between user input tokens and product tokens
        for user_token in user_input_tokens:
            for product_token in product_tokens:
                similarity_score = fuzz.ratio(user_token, product_token)
                
                # Set a threshold for similarity scores
                if similarity_score > 90:
                    matching_products.append({
                        'product_id': row['product_id'],
                        'rental_place_id': row['rental_place_id']
                    })
                    break  # Move to the next product
            
            if matching_products:  # Break out of the loop if a match is found
                break
    
    return matching_products

# Retrieve detailed rental place information based on rental_place_id

def get_rental_place_info(rental_place_id):
    
    rental_place_info = rentaldf[rentaldf['rental_place_id'] == str(rental_place_id)][['rental_place_name', 'address', 'email','phone','website']]
    
    return rental_place_info

# Function to process user input and find matching products

def process_user_input(user_input_list, products_dataframes):
    
    for i in range(len(user_input_list)):
        tokens = user_input_list[i].split(' ')

    # Iterate through each product DataFrame
    for df in products_dataframes:

        matching_products = find_matching_products(tokens, df['tokens'])
        
        if matching_products:
            for product in matching_products:
                rental_place_id = product['rental_place_id']
                rental_place_info = get_rental_place_info(rental_place_id)
                
                # Display or store rental place information
                return rental_place_info
                # You can modify this part to return